In [10]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm 
from scipy import stats
import itertools
from iminuit import Minuit   
from scipy.optimize import minimize
import sys  
from importlib import reload
from scipy import interpolate
import copy

# external libraries
sys.path.append('../') 
import AdvAppStatFunctions as aas

# setting for plotting
import seaborn as sns
plt.style.use('seaborn-white')
sns.set_style('white', {'legend.frameon':True})
plt.rcParams['font.size'] = 14
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
sns.set_palette("colorblind")
color = sns.color_palette("colorblind")

from sklearn.model_selection import cross_val_score
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.optim import Adam
from tqdm.auto import trange

In [36]:
# Split data into training and test set
X_train = np.genfromtxt('Set3_Prob4_TrainData.csv', skip_header=1, delimiter=',')[:,1:-1]
Y_train = np.genfromtxt('Set3_Prob4_trainData.csv', skip_header=1, delimiter=',')[:,-1]
print(X_train.shape, Y_train.shape)

X_test = np.genfromtxt('Set3_Prob4_TestData.csv', skip_header=1, delimiter=',')[:,1:-1]
Y_test = np.genfromtxt('Set3_Prob4_TestData.csv', skip_header=1, delimiter=',')[:,-1]

# Further split test set into test and validation sets
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5)

# Look at lengths of data
n_train = len(X_train)
n_val = len(X_val)
n_test = len(X_test)
print(f"{n_train} training samples")
print(f"{n_val} validation samples")
print(f"{n_test} test samples")

# Convert to tensors
#X_train = torch.as_tensor(X_train)
#X_val = torch.as_tensor(X_val)
#X_test = torch.as_tensor(X_test)
#Y_train = torch.as_tensor(Y_train)
#Y_val = torch.as_tensor(Y_val)
#Y_test = torch.as_tensor(Y_test)


X_train = torch.FloatTensor(X_train)
Y_train = torch.LongTensor(Y_train)
X_val = torch.FloatTensor(X_val)
Y_val = torch.LongTensor(Y_val)
X_test = torch.FloatTensor(X_test)
Y_test = torch.LongTensor(Y_test)

(3000, 15) (3000,)
3000 training samples
1500 validation samples
1500 test samples


In [33]:
X_train

tensor([[  0.,   0.,  18.,  ...,   2.,   1.,   0.],
        [  0.,   0.,  53.,  ...,   3.,   0.,   0.],
        [  0.,   0.,  17.,  ...,   2.,   0.,   0.],
        ...,
        [  0.,   0.,  54.,  ...,   3.,   0.,   0.],
        [  0.,   0.,   2.,  ...,   4.,   0.,   1.],
        [  0.,   0., 189.,  ...,   2.,   0.,   0.]])

In [35]:
# Make a simple neural network. We'll train this to output the *unnormalized* log-probabilities
# that the input colors correspond to a galaxy and quasar respectively.
hidden_dim = 8
network = nn.Sequential(
    nn.Linear(15, hidden_dim),  # The size of the input layer should match the number of features in your input dat
    nn.ReLU(),
    nn.Linear(hidden_dim, hidden_dim),
    nn.ReLU(),
    nn.Linear(hidden_dim, 2), # The size of the output layer should match the number of classes you're trying to predict
    #nn.Softmax(dim=1)
)

n_net_params = sum([p.numel() for p in network.parameters()])
print(f"Network contains {n_net_params} parameters")

# Set optimizer, loss and other training parameters
optimizer = Adam(network.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss(reduction="sum")
batch_size = 100 #100
n_epochs = 15

# For monitoring losses
avg_epoch_losses_train = []
avg_epoch_losses_val = [np.inf]
pbar = trange(n_epochs)

# Train!
print("Training...")
for i in pbar:  # progress bar
    # Training
    optimizer.zero_grad()
    network.train()
    epoch_loss = 0
    for x, y in zip(X_train[::batch_size], Y_train[::batch_size]):
        predicted_logprobs = network(x)
        #loss = loss_fn(predicted_logprobs, y)
        loss = loss_fn(predicted_logprobs, torch.argmax(y, dim=1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    avg_epoch_losses_train.append(epoch_loss / (n_train // batch_size))

    # Validation
    network.eval()
    epoch_loss = 0
    with torch.no_grad():
        for x, y in zip(X_val[::batch_size], Y_val[::batch_size]):
            predicted_logprobs = network(x)
            loss = loss_fn(predicted_logprobs, y)
            epoch_loss += loss.item()
        avg_epoch_losses_val.append(epoch_loss / (n_val // batch_size))

print("Done!")

Network contains 218 parameters


  0%|          | 0/15 [00:00<?, ?it/s]

Training...


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)